In [1]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
import json
import pytz
import uuid
from datetime import datetime
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from flatten_json import flatten

import os
import pandas as pd
import math

# Set pandas dataframe display formatting
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

#Load LUSID API Components
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)

quotes_api = api_factory.build(lusid.api.QuotesApi) # market data for analytics
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi) # market data for analytics

In [2]:
scope = "curve_bootstrapping"
base_date = datetime(2012, 12, 13, tzinfo=pytz.utc)

In [3]:
 def create_fra(start, maturity, rate):
    return models.ForwardRateAgreement(
                start_date = start.isoformat(),
                maturity_date = maturity.isoformat(), 
                dom_ccy = "EUR",
                fixing_date = maturity.isoformat(), 
                fra_rate = rate,
                notional = 10000000,
                instrument_type="ForwardRateAgreement"
            )

fra_data = [
    (datetime(2012, 12, 13, tzinfo=pytz.utc), datetime(2013, 3, 13, tzinfo=pytz.utc), .00181),
    (datetime(2012, 12, 14, tzinfo=pytz.utc), datetime(2013, 3, 14, tzinfo=pytz.utc), .00179),
    (datetime(2013, 1, 14, tzinfo=pytz.utc), datetime(2013, 4, 15, tzinfo=pytz.utc), .00165),
    (datetime(2013, 2, 13, tzinfo=pytz.utc), datetime(2013, 5, 13, tzinfo=pytz.utc), .00141),
    (datetime(2013, 3, 13, tzinfo=pytz.utc), datetime(2013, 6, 13, tzinfo=pytz.utc), .00129),
    (datetime(2013, 4, 15, tzinfo=pytz.utc), datetime(2013, 7, 15, tzinfo=pytz.utc), .00126),
    (datetime(2013, 5, 13, tzinfo=pytz.utc), datetime(2013, 8, 13, tzinfo=pytz.utc), .00124),
    (datetime(2013, 6, 13, tzinfo=pytz.utc), datetime(2013, 9, 13, tzinfo=pytz.utc), .00121),
]

fras = [create_fra(start, end, rate) for (start, end, rate) in fra_data]

In [11]:
def create_swap(start, maturity, fixed_rate):
    flow_conventionFixed = models.FlowConventions(
        currency="EUR",
        payment_frequency="3M",
        day_count_convention="Act360",
        roll_convention="ModifiedFollowing",
        settle_days=2,
        reset_days=2,
        payment_calendars=[],
        reset_calendars=[]
    )
    
    flow_conventionFloat = models.FlowConventions(
        currency="EUR",
        payment_frequency="3M",
        day_count_convention="Act360",
        roll_convention="ModifiedFollowing",
        settle_days=2,
        reset_days=2,
        payment_calendars=[],
        reset_calendars=[]
    )

    # For projection rates it requires:
    # ccy/tenor/indexname dependency
    # For past data:
    # Looks for quotes in quote store with reference = fixingreference
    float_leg_idx_conv = models.IndexConvention(
        currency="EUR",
        payment_tenor="3M",
        fixing_reference="EURIBOR3M",
        publication_day_lag=0,
        day_count_convention="Act360",
        index_name="blah"
    )
    
    # Create the leg definitions
    float_leg_definition = models.LegDefinition(
        index_convention=float_leg_idx_conv,
        pay_receive="Receive",
        conventions=flow_conventionFloat,
        stub_type="Both",
        notional_exchange_type="None",
        rate_or_spread=0.0,
        reset_convention="InArrears",
    )

    fixed_leg_definition = models.LegDefinition(
        rate_or_spread=fixed_rate,
        pay_receive="Pay",
        conventions=flow_conventionFixed,
        stub_type="Both",
        notional_exchange_type="None",
    )

    # Create the legs
    fixed_leg = models.FixedLeg(
        start_date=start.isoformat(),
        maturity_date=maturity.isoformat(),
        notional=10000000,
        leg_definition=fixed_leg_definition,
        instrument_type="FixedLeg",
        overrides={}
    )

    floating_leg = models.FloatingLeg(
        start_date=start.isoformat(),
        maturity_date=maturity.isoformat(),
        notional=10000000,
        leg_definition=float_leg_definition,
        instrument_type="FloatingLeg",
        overrides={}
    )

    irs_legs = [
        fixed_leg,
        floating_leg
    ]

    # create the swap
    instrument_definition_IRS = models.InterestRateSwap(
        start_date=start.isoformat(),
        maturity_date=maturity.isoformat(),
        legs=irs_legs,
        instrument_type="InterestRateSwap")

   
    return instrument_definition_IRS

swap_data = [
    (datetime(2013, 12, 13, tzinfo=pytz.utc), .00141),
    (datetime(2014, 3, 13, tzinfo=pytz.utc), .00144),
    (datetime(2014, 6, 13, tzinfo=pytz.utc), .00153),
    (datetime(2014, 9, 15, tzinfo=pytz.utc), .00168),
    (datetime(2014, 12, 15, tzinfo=pytz.utc), .00186),
    (datetime(2015, 12, 14, tzinfo=pytz.utc), .00285),
    (datetime(2016, 12, 13, tzinfo=pytz.utc), .00437),
    (datetime(2017, 12, 13, tzinfo=pytz.utc), .00623),
    (datetime(2018, 12, 13, tzinfo=pytz.utc), .00817),
    (datetime(2019, 12, 13, tzinfo=pytz.utc), .01000),
    (datetime(2020, 12, 14, tzinfo=pytz.utc), .01171),
    (datetime(2021, 12, 13, tzinfo=pytz.utc), .01324),
    (datetime(2022, 12, 13, tzinfo=pytz.utc), .01459),
    (datetime(2023, 12, 13, tzinfo=pytz.utc), .01582),
    (datetime(2024, 12, 13, tzinfo=pytz.utc), .01692),
    (datetime(2027, 12, 13, tzinfo=pytz.utc), .01933),
    (datetime(2032, 12, 13, tzinfo=pytz.utc), .02099),
    (datetime(2037, 12, 14, tzinfo=pytz.utc), .02156),
    (datetime(2042, 12, 15, tzinfo=pytz.utc), .02186),
    (datetime(2052, 12, 13, tzinfo=pytz.utc), .02288),
    (datetime(2062, 12, 13, tzinfo=pytz.utc), .02367),
]

swaps = [create_swap(base_date, maturity, rate) for (maturity, rate) in swap_data]

In [12]:
def quote_from_value(value: float):
        return models.MarketQuote(models.QuoteType.RATE, value)
quote_values = [fra.fra_rate for fra in fras] + [swap.legs[0].leg_definition.rate_or_spread for swap in swaps]

yield_curve = models.YieldCurveData(
        base_date,
        instruments = fras + swaps,
        quotes = list(map(quote_from_value, quote_values)),
        market_data_type = models.MarketDataType.YIELDCURVEDATA
    )

In [6]:
def upsert_complex_market_data(complex_market_data, asset_name):
    complex_id = models.ComplexMarketDataId(provider="Lusid",
                                            effective_at=base_date,
                                            market_asset=asset_name)

    upsert_request = models.UpsertComplexMarketDataRequest(market_data_id=complex_id,
                                                              market_data=complex_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertComplexMarketData
    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope,
        request_body={"1": upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload yield curve {response.failed}")

    print(f"Complex market data {asset_name} uploaded into scope={scope}")
    
upsert_complex_market_data(yield_curve, "EUR/3M/blah") # projection curve
upsert_complex_market_data(yield_curve, "EUR/EUROIS") # identical discounting curve

Complex market data EUR/3M/blah uploaded into scope=curve_bootstrapping
Complex market data EUR/EUROIS uploaded into scope=curve_bootstrapping


In [7]:
# create recipe that prices everything via LUSID's native pricing library
lusid_recipe_code = "LusidRecipe-All"

lusid_pricing_context = models.PricingContext(
    options=models.PricingOptions(
        model_selection=models.ModelSelection(
            library="Lusid",
            model="Discounting"
        )
    )
)

def upsert_recipe(recipe_code, pricing_context):
    lusid_market_context = models.MarketContext(
        market_rules = [],
        options=models.MarketOptions(
            default_supplier="Lusid",
            default_scope=scope)
    )

    recipe = models.ConfigurationRecipe(
            scope=scope,
            code=recipe_code,
            description=recipe_code,
            market=lusid_market_context,
            pricing=pricing_context
    )
    
    # Upsert recipe to LUSID
    upsert_recipe_request = models.UpsertRecipeRequest(configuration_recipe=recipe)
    configuration_recipe_api.upsert_configuration_recipe(upsert_recipe_request)
    print(f"Recipe {recipe_code} updated in scope {scope}")
    
upsert_recipe(lusid_recipe_code, lusid_pricing_context)

Recipe LusidRecipe-All updated in scope curve_bootstrapping


In [26]:
def upsert_instrument_to_lusid(instrument_definition, name, identifier):
        response = instruments_api.upsert_instruments(
            request_body={
                identifier: models.InstrumentDefinition(
                    name=name,
                    identifiers={
                        "ClientInternal": models.InstrumentIdValue(value=identifier)
                    },
                    definition=instrument_definition,
                )
            }
        )
        luid = response.values[identifier].lusid_instrument_id
        print(f"{name} = " + luid)
        return luid
    
def instrument_defn_to_weighted_instrument(instrument_defn):
    return models.WeightedInstrument(
        quantity=1,
        holding_identifier=f"some_instrument",
        instrument=instrument_defn)
    
def generate_valuation_request(instrument_defns):
    if type(instrument_defns) is list:
        weighted_instruments = list(map(instrument_defn_to_weighted_instrument, instrument_defns))
    else:
        weighted_instruments = [instrument_defn_to_weighted_instrument(instrument_defns)]

    # Create the valuation request
    valuation_request = models.InlineValuationRequest(
        recipe_id=models.ResourceId(
            scope=scope, code="LusidRecipe-All"
        ),
        metrics=[
            models.AggregateSpec("Instrument/InstrumentType", "Value"),
            models.AggregateSpec("Instrument/Definition/StartDate", "Value"),
            models.AggregateSpec("Instrument/Definition/MaturityDate", "Value"),
            models.AggregateSpec("Valuation/PV", "Value"),
        ],
        report_currency = "EUR",
        valuation_schedule=models.ValuationSchedule(
            effective_at=base_date.isoformat()
        ),
        instruments=weighted_instruments
    )

    return valuation_request

In [29]:
val_req = generate_valuation_request(swaps) # construct or choose an instrument to price

aggregation = aggregation_api.get_valuation_of_weighted_instruments(inline_valuation_request=val_req)

output = pd.DataFrame(aggregation.data)
output

,Instrument/InstrumentType,Instrument/Definition/StartDate,Instrument/Definition/MaturityDate,Valuation/PV
0,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2013-12-13T00:00:00.0000000+00:00,-612.11
1,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2014-03-13T00:00:00.0000000+00:00,420.71
2,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2014-06-13T00:00:00.0000000+00:00,"1,887.99"
3,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2014-09-15T00:00:00.0000000+00:00,"3,388.54"
4,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2014-12-15T00:00:00.0000000+00:00,"5,852.63"
5,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2015-12-14T00:00:00.0000000+00:00,"15,263.22"
6,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2016-12-13T00:00:00.0000000+00:00,"26,756.76"
7,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2017-12-13T00:00:00.0000000+00:00,"37,457.70"
8,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2018-12-13T00:00:00.0000000+00:00,"45,796.61"
9,InterestRateSwap,2012-12-13T00:00:00.0000000+00:00,2019-12-13T00:00:00.0000000+00:00,"52,967.79"
